#### Dataset Creation

In [1]:
from data_loading.models_dataset import EcoreModelDataset, ArchiMateModelDataset

config_params = dict(
    min_edges = -1,
    reload=True,
)
ecore_model_dataset = EcoreModelDataset('ecore_555', **config_params)
# ecore_model_dataset = EcoreModelDataset('mar-ecore-github', reload=False, **config_params)
# ecore_model_dataset = EcoreModelDataset('modelset', reload=False, **config_params)
# eamodelset = ArchiMateModelDataset('eamodelset', **config_params)

Loading Ecore_555:   0%|          | 0/548 [00:00<?, ?it/s]

Loaded Total ecore_555 with 548 graphs
Filtering...
Saving ecore_555 to pickle
Saved ecore_555 to pickle
Loaded ecore_555 with 548 graphs


In [2]:
from data_loading.models_dataset import EcoreModelDataset

dataset_name = 'ecore_555'

dataset = EcoreModelDataset(dataset_name, min_edges=10, reload=True)

Loading Ecore_555:   0%|          | 0/548 [00:00<?, ?it/s]

Loaded Total ecore_555 with 548 graphs
Filtering...
Saving ecore_555 to pickle
Saved ecore_555 to pickle
Loaded ecore_555 with 444 graphs


In [3]:
from data_loading.graph_dataset import GraphNodeDataset


graph_dataset = GraphNodeDataset(dataset)

Creating graphs:   0%|          | 0/444 [00:00<?, ?it/s]

Processing graphs:   0%|          | 0/444 [00:00<?, ?it/s]

Node label: abstract
Train Node classes: {0: 10577, 1: 3212}
Test Node classes: {0: 2806, 1: 856}


In [4]:
lm_dataset = graph_dataset.get_lm_graph_classification_data()

In [5]:
from utils import set_seed


set_seed(42)

gnn_data = graph_dataset.get_gnn_graph_classification_data()

In [6]:
len(gnn_data['train'])

355

In [7]:
set_seed(42)

lm_data = graph_dataset.get_lm_graph_classification_data()


In [8]:
gnn_y = [g.graph_label.item() for g in gnn_data['train']]

In [9]:
lm_y = [l.item() for l in lm_data['train'][:]['labels']]

In [10]:
print(lm_y)

[7, 6, 4, 6, 6, 5, 5, 3, 7, 0, 4, 3, 4, 5, 4, 4, 4, 2, 1, 4, 6, 4, 5, 6, 0, 3, 7, 5, 6, 1, 1, 3, 5, 6, 5, 2, 6, 3, 4, 4, 6, 7, 4, 6, 5, 6, 6, 7, 2, 4, 3, 0, 4, 6, 6, 6, 1, 3, 3, 0, 6, 2, 6, 6, 4, 1, 2, 1, 5, 7, 5, 6, 2, 3, 3, 3, 6, 4, 3, 7, 1, 0, 1, 5, 6, 4, 6, 6, 5, 0, 1, 2, 6, 6, 0, 3, 6, 6, 4, 5, 6, 6, 6, 6, 6, 3, 2, 5, 6, 1, 4, 3, 0, 0, 6, 3, 6, 6, 3, 5, 4, 0, 3, 4, 6, 3, 6, 6, 6, 5, 6, 5, 5, 6, 4, 5, 5, 7, 4, 0, 0, 4, 6, 3, 3, 5, 6, 7, 5, 5, 3, 0, 6, 4, 6, 6, 6, 5, 3, 2, 0, 6, 3, 1, 5, 6, 1, 6, 7, 0, 1, 3, 4, 6, 6, 4, 1, 1, 5, 0, 3, 3, 7, 0, 5, 6, 4, 7, 6, 6, 1, 0, 5, 3, 3, 4, 5, 2, 6, 3, 3, 7, 5, 0, 4, 5, 7, 5, 6, 0, 5, 6, 4, 5, 6, 3, 0, 6, 5, 6, 0, 6, 3, 6, 4, 7, 3, 7, 1, 7, 6, 5, 3, 6, 6, 6, 5, 3, 0, 3, 3, 6, 3, 3, 2, 3, 6, 0, 6, 1, 4, 3, 2, 0, 0, 4, 6, 4, 4, 5, 3, 6, 4, 6, 0, 6, 4, 5, 6, 3, 1, 7, 2, 7, 3, 4, 6, 6, 0, 0, 6, 6, 1, 5, 3, 2, 6, 6, 3, 6, 0, 0, 4, 3, 6, 1, 6, 4, 0, 0, 3, 5, 3, 7, 3, 3, 6, 6, 6, 5, 4, 5, 6, 0, 6, 0, 5, 3, 0, 0, 3, 5, 4, 6, 3, 4, 1, 4, 2, 6, 6, 5, 3, 

In [11]:
len(gnn_y), len(lm_y)

(355, 355)

In [12]:
lm_y == gnn_y

True